Alzheimer's Detection Using CNN and MRI Scans

Objective:

Develop a Convolutional Neural Network (CNN) to detect Alzheimer's disease from MRI
brain scans by classifying images into different categories such as Mild Cognitive
Impairment (MCI), Alzheimer’s Disease (AD), and Normal Controls (NC).

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
import numpy as np
import os

In [4]:
folder=r'D:\soft computing\project\Axial\AD'
j=os.listdir(folder)
img=cv2.imread(os.path.join(folder,j[0]))
img.shape

(256, 170, 3)

In [72]:
X_train = []
Y_train = []
image_size = 64 #As we are using AlexNet, the standard size is 227 x 227
labels = ['AD','CN','MCI']
for i in labels:
    folderPath = os.path.join(r'D:\soft computing\project\Axial',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        # img=cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        X_train.append(img)
        Y_train.append(i)
        
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [73]:
X_train.shape

(5154, 64, 64, 3)

In [74]:
X_train,Y_train = shuffle(X_train,Y_train,random_state=101)
x_train,x_test,y_train,y_test = train_test_split(X_train,Y_train,test_size=0.55,random_state=101)

In [106]:
x_test_data1,x_test_data2,y_test_data1,y_test_data2=train_test_split(x_test,y_test,test_size=0.7,random_state=101)

In [109]:
x_test_data1.shape

(850, 64, 64, 3)

In [75]:
x_train.shape

(2319, 64, 64, 3)

In [76]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train=np.eye(len(labels))[y_train_new]

y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test=np.eye(len(labels))[y_test_new]

In [77]:
print(x_train.shape)
y_train.shape

(2319, 64, 64, 3)


(2319, 3)

In [ ]:
import numpy as np

# Define a simple 2D Convolution class
class SimpleConv2D:
    def __init__(self, input_shape, filter_size, num_filters, stride=1, padding='same'):
        self.input_shape = input_shape  # Shape of the input image (height, width, channels)
        self.filter_size = filter_size  # Size of the convolutional filter (height, width)
        self.num_filters = num_filters  # Number of filters (output depth)
        self.stride = stride  # Stride of the convolution operation
        self.padding = padding  # Padding method ('same' or 'valid')
        
        # Initialize filters with random values and biases with zeros
        self.filters = np.random.randn(filter_size[0], filter_size[1], input_shape[2], num_filters)
        self.bias = np.zeros((num_filters,))
    
    # Define the convolution operation
    def conv2d(self, input_image):
        input_height, input_width, input_channels = self.input_shape
        
        # Apply padding if 'same' padding is specified
        if self.padding == 'same':
            pad_height = (self.filter_size[0] - 1) // 2
            pad_width = (self.filter_size[1] - 1) // 2
            input_image = np.pad(input_image, ((pad_height, pad_height), (pad_width, pad_width), (0, 0)), mode='constant')
            input_height, input_width = input_image.shape[0], input_image.shape[1]
        
        # Calculate the dimensions of the output
        output_height = (input_height - self.filter_size[0]) // self.stride + 1
        output_width = (input_width - self.filter_size[1]) // self.stride + 1
        
        # Initialize the output feature map
        output = np.zeros((output_height, output_width, self.num_filters))
        
        # Perform the convolution operation
        for i in range(output_height):
            for j in range(output_width):
                # Extract the patch of the input image over which the filter slides
                image_patch = input_image[i*self.stride:i*self.stride+self.filter_size[0], 
                                          j*self.stride:j*self.stride+self.filter_size[1], :]
                for k in range(self.num_filters):
                    # Convolve the filter with the image patch and add the bias
                    output[i, j, k] = np.sum(image_patch * self.filters[:, :, :, k]) + self.bias[k]
        
        return output

# Define a simple Max Pooling class
class SimpleMaxPooling:
    def __init__(self, pool_size, stride):
        self.pool_size = pool_size  # Size of the pooling window
        self.stride = stride  # Stride of the pooling operation
    
    # Perform max pooling on the input image
    def max_pool(self, input_image):
        input_height, input_width, input_channels = input_image.shape
        
        # Calculate the dimensions of the output
        output_height = (input_height - self.pool_size) // self.stride + 1
        output_width = (input_width - self.pool_size) // self.stride + 1
        
        # Initialize the output
        output = np.zeros((output_height, output_width, input_channels))
        
        # Perform max pooling
        for i in range(output_height):
            for j in range(output_width):
                for c in range(input_channels):
                    # Extract the patch of the input image
                    region = input_image[i*self.stride:i*self.stride+self.pool_size, 
                                         j*self.stride:j*self.stride+self.pool_size, c]
                    # Compute the maximum value in the patch
                    output[i, j, c] = np.max(region)
        
        return output

# Define the ReLU activation function
def relu(x):
    return np.maximum(0, x)

# Flatten the 3D input into a 1D array
def flatten(input_image):
    return input_image.flatten()

# Define the AlexNet architecture with Dropout
def alexnet(input_image):
    # Layer 1: Conv1 (96 filters, 11x11, stride=4, padding='same') + ReLU + MaxPooling
    conv1 = SimpleConv2D(input_shape=(64, 64, 3), filter_size=(4, 4), num_filters=96, stride=4, padding='same')
    out1 = conv1.conv2d(input_image)  # Convolution operation
    out1 = relu(out1)  # Apply ReLU activation
    
    maxpool1 = SimpleMaxPooling(pool_size=2, stride=2)  # Define max pooling layer
    out1 = maxpool1.max_pool(out1)  # Apply max pooling
    
    # Layer 2: Conv2 (256 filters, 5x5, stride=1, padding='same') + ReLU + MaxPooling
    conv2 = SimpleConv2D(input_shape=out1.shape, filter_size=(3, 3), num_filters=256, stride=2 , padding='same')
    out2 = conv2.conv2d(out1)  # Convolution operation
    out2 = relu(out2)  # Apply ReLU activation
    
    maxpool2 = SimpleMaxPooling(pool_size=2, stride=1)  # Define max pooling layer
    out2 = maxpool2.max_pool(out2)  # Apply max pooling
    
    # Layer 3: Conv3 (384 filters, 3x3, stride=1, padding='same') + ReLU
    conv3 = SimpleConv2D(input_shape=out2.shape, filter_size=(3, 3), num_filters=384, stride=1, padding='same')
    out3 = conv3.conv2d(out2)  # Convolution operation
    out3 = relu(out3)  # Apply ReLU activation
    
    # Layer 4: Conv4 (384 filters, 3x3, stride=1, padding='same') + ReLU
    conv4 = SimpleConv2D(input_shape=out3.shape, filter_size=(3, 3), num_filters=384, stride=1, padding='same')
    out4 = conv4.conv2d(out3)  # Convolution operation
    out4 = relu(out4)  # Apply ReLU activation
    
    # Layer 5: Conv5 (256 filters, 3x3, stride=1, padding='same') + ReLU + MaxPooling
    conv5 = SimpleConv2D(input_shape=out4.shape, filter_size=(3, 3), num_filters=256, stride=1, padding='same')
    out5 = conv5.conv2d(out4)  # Convolution operation
    out5 = relu(out5)  # Apply ReLU activation
    
    maxpool3 = SimpleMaxPooling(pool_size=2, stride=1)  # Define max pooling layer
    out5 = maxpool3.max_pool(out5)  # Apply max pooling
    
    # Flatten the output for fully connected layers
    flattened = flatten(out5)
    
    # Apply dropout to the flattened output
    # flattened = dropout(flattened, drop_probability=dropout_probability)
    return flattened
# Example input (randomly initialized 227x227x3 image)
result = []
for img in x_train:
    # Process each input image through the AlexNet model
    temp = alexnet(img)
    result.append(temp)

# Convert the results to a numpy array
result = np.array(result)
print("Final Feature Shape:", result.shape)

Final Feature Shape: (2319, 1024)


In [110]:
x_test_flat=[]
for img in x_test_data1:
    temp=alexnet(img)
    x_test_flat.append(temp)
x_test_flat=np.array(x_test_flat)
print("Final size of test data:",x_test_flat.shape)

Final size of test data: (850, 1024)


In [96]:
result.shape
x_train_flat=result
print(x_train_flat.shape)

(2319, 1024)


Bpn with Relu and softmax functions

In [119]:
import numpy as np

# Activation functions
def relu(x):
    """ReLU activation function: returns max(0, x) for each element in x."""
    return np.maximum(0, x)

def relu_derivative(x):
    """Derivative of the ReLU activation function: returns 1 for x > 0, and 0 otherwise."""
    return (x > 0).astype(float)

def softmax(x):
    """Softmax activation function: normalizes the input into probabilities."""
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))  # Subtract max(x) for numerical stability
    return exps / np.sum(exps, axis=1, keepdims=True)  # Normalize to get probabilities

# Cross-entropy loss function
def cross_entropy_loss(y_true, y_pred):
    """Cross-entropy loss function for multi-class classification."""
    epsilon = 1e-12  # Small epsilon to avoid log(0) which results in NaN
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)  # Clip values to avoid extreme log values
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))  # Compute average loss

# Model initialization
def initialize_weights(input_size, hidden_size, output_size):
    """Initialize weights and biases for the model."""
    W1 = np.random.randn(input_size, hidden_size) * 0.01  # Small random values for input to hidden weights
    b1 = np.zeros((1, hidden_size))  # Biases for the hidden layer initialized to zeros
    W2 = np.random.randn(hidden_size, output_size) * 0.01  # Small random values for hidden to output weights
    b2 = np.zeros((1, output_size))  # Biases for the output layer initialized to zeros
    return W1, b1, W2, b2

# Forward propagation
def forward_propagation(X, W1, b1, W2, b2):
    """Perform forward propagation to compute predictions."""
    Z1 = np.dot(X, W1) + b1  # Compute input to the hidden layer (Z1)
    A1 = relu(Z1)  # Apply ReLU activation to hidden layer input (A1)
    Z2 = np.dot(A1, W2) + b2  # Compute input to the output layer (Z2)
    A2 = softmax(Z2)  # Apply Softmax to get output probabilities (A2)
    return Z1, A1, Z2, A2  # Return intermediate values for use in backward propagation

# Backward propagation
def backward_propagation(X, Y, Z1, A1, A2, W2):
    """Compute gradients during backward propagation."""
    m = X.shape[0]  # Number of training examples (samples)

    # Output layer gradients (dW2, db2)
    dZ2 = A2 - Y  # Derivative of loss with respect to output (cross-entropy)
    dW2 = np.dot(A1.T, dZ2) / m  # Gradient of the weights for the output layer
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m  # Gradient of the biases for the output layer

    # Hidden layer gradients (dW1, db1)
    dA1 = np.dot(dZ2, W2.T)  # Gradient with respect to hidden layer's activations
    dZ1 = dA1 * relu_derivative(Z1)  # Derivative of ReLU for hidden layer
    dW1 = np.dot(X.T, dZ1) / m  # Gradient of the weights for the hidden layer
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m  # Gradient of the biases for the hidden layer

    return dW1, db1, dW2, db2  # Return all gradients for weight and bias updates

# Training the model
def train(X_train, Y_train, input_size, hidden_size, output_size, epochs, learning_rate):
    """Train the neural network model using forward and backward propagation."""
    # Initialize model weights and biases
    W1, b1, W2, b2 = initialize_weights(input_size, hidden_size, output_size)

    for epoch in range(epochs):
        # Forward pass to compute predictions
        Z1, A1, Z2, A2 = forward_propagation(X_train, W1, b1, W2, b2)

        # Compute the loss (cross-entropy)
        loss = cross_entropy_loss(Y_train, A2)

        # Backward pass to compute gradients
        dW1, db1, dW2, db2 = backward_propagation(X_train, Y_train, Z1, A1, A2, W2)

        # Update weights and biases using the gradients
        W1 -= learning_rate * dW1  # Update weights for the hidden layer
        b1 -= learning_rate * db1  # Update biases for the hidden layer
        W2 -= learning_rate * dW2  # Update weights for the output layer
        b2 -= learning_rate * db2  # Update biases for the output layer

        # Print loss every 10 epochs or at the first epoch
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    return W1, b1, W2, b2  # Return the trained weights and biases

# Prediction function
def predict(X, W1, b1, W2, b2):
    """Make predictions for the input data X using the trained model."""
    _, _, _, A2 = forward_propagation(X, W1, b1, W2, b2)  # Perform forward propagation
    return np.argmax(A2, axis=1)  # Return the index of the maximum probability (the predicted class)

# Example usage
if __name__ == "__main__":
    # Data dimensions for a dataset with 1024 input features and 3 output classes
    input_size = 1024  # Number of features (input layer size)
    hidden_size = 512  # Number of neurons in the hidden layer
    output_size = 3  # Number of classes in the output layer (Alzheimer classification with 3 classes)

    # Synthetic data for testing (replace with actual data)
    X_train = result  # Training data
    Y_train = y_train  # One-hot encoded labels for training data

    # Hyperparameters for training
    epochs = 50  # Number of epochs to train the model
    learning_rate = 0.01  # Learning rate for weight updates

    # Train the model and obtain the final weights and biases
    W1, b1, W2, b2 = train(X_train, Y_train, input_size, hidden_size, output_size, epochs, learning_rate)

    # Make predictions on the training data (or test data)
    predictions = predict(X_train, W1, b1, W2, b2)

    # Calculate the training accuracy by comparing predicted classes with true classes
    accuracy = np.mean(predictions == y_train_new)  # y_train_new contains the actual class labels
    print(f"Training Accuracy: {accuracy:.4f}")  # Output the accuracy


Epoch 1/50, Loss: 19.3977
Epoch 10/50, Loss: 1.0959
Epoch 20/50, Loss: 1.0920
Epoch 30/50, Loss: 1.0883
Epoch 40/50, Loss: 1.0849
Epoch 50/50, Loss: 1.0816
Training Accuracy: 0.4955


In [120]:
# Function to predict class labels
def predict_relu_softmax(X):
    # Forward pass
    z1 = np.dot(X, W1) + b1  # Hidden layer input
    a1 = relu(z1)  # Hidden layer output (ReLU activation)
    z2 = np.dot(a1, W2) + b2  # Output layer input
    a2 = softmax(z2)  # Output layer output (Softmax activation)
    return a2

# Function to calculate accuracy
def calculate_accuracy_relu_softmax(X_test, Y_test):
    # Get predictions
    predictions = predict_relu_softmax(X_test)
    
    # Convert predictions to class labels
    predicted_classes = np.argmax(predictions, axis=1)  # Choose the class with the highest probability
    true_classes = np.argmax(Y_test, axis=1)  # Convert one-hot encoded labels to class labels
    
    # Calculate accuracy
    accuracy = np.mean(predicted_classes == true_classes)  # Percentage of correct predictions
    return accuracy

# Example usage:
accuracy_relu_softmax = calculate_accuracy_relu_softmax(x_test_flat, y_test_data1)  # X_test_flat and y_test are your testing data and labels
print(f"Accuracy for ReLU + Softmax Network: {accuracy_relu_softmax * 100:.2f}%")


Accuracy for ReLU + Softmax Network: 49.88%


Pure Bpn (Uses sigmoid function)

In [121]:
import numpy as np

# Activation functions

def sigmoid(x):
    """Sigmoid activation function: computes 1 / (1 + exp(-x)) with numerical stability."""
    x = np.clip(x, -15, 15)  # Clip the input to avoid overflow issues in exp
    return 1 / (1 + np.exp(-x))  # Sigmoid function: squashes input to range (0, 1)

def sigmoid_derivative(x):
    """Derivative of the sigmoid activation function."""
    return x * (1 - x)  # Sigmoid derivative, using the output of sigmoid function

# Cross-entropy loss function
def cross_entropy_loss(y_true, y_pred):
    """Computes the cross-entropy loss, which is used for multi-class classification."""
    epsilon = 1e-12  # Small epsilon to avoid log(0) which can result in NaN
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)  # Clip predicted values to avoid log(0)
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))  # Cross-entropy loss calculation

# Model initialization function
def initialize_weights_bpn(input_size, hidden_size, output_size):
    """Initializes the weights and biases for the model."""
    W1 = np.random.randn(input_size, hidden_size) * 0.01  # Random weights for input to hidden layer
    b1 = np.zeros((1, hidden_size))  # Initialize biases for the hidden layer to zeros
    W2 = np.random.randn(hidden_size, output_size) * 0.01  # Random weights for hidden to output layer
    b2 = np.zeros((1, output_size))  # Initialize biases for the output layer to zeros
    return W1, b1, W2, b2  # Return the initialized weights and biases

# Forward propagation function
def forward_propagation_bpn(X, W1, b1, W2, b2):
    """Performs forward propagation through the network."""
    Z1 = np.dot(X, W1) + b1  # Linear combination for input to hidden layer
    A1 = sigmoid(Z1)  # Apply sigmoid activation to the hidden layer
    Z2 = np.dot(A1, W2) + b2  # Linear combination for hidden to output layer
    A2 = sigmoid(Z2)  # Apply sigmoid activation to the output layer
    return Z1, A1, Z2, A2  # Return intermediate values for use in backward propagation

# Backward propagation function
def backward_propagation_bpn(X, Y, Z1, A1, A2, W2):
    """Performs backward propagation to compute gradients."""
    m = X.shape[0]  # Number of training samples

    # Output layer gradients (dW2, db2)
    dZ2 = A2 - Y  # Derivative of the loss w.r.t output (cross-entropy)
    dW2 = np.dot(A1.T, dZ2) / m  # Gradient of weights from hidden to output layer
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m  # Gradient of biases for output layer

    # Hidden layer gradients (dW1, db1)
    dA1 = np.dot(dZ2, W2.T)  # Gradient w.r.t hidden layer's activations
    dZ1 = dA1 * sigmoid_derivative(A1)  # Derivative of the sigmoid activation for hidden layer
    dW1 = np.dot(X.T, dZ1) / m  # Gradient of weights from input to hidden layer
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m  # Gradient of biases for hidden layer

    return dW1, db1, dW2, db2  # Return the computed gradients

# Training function
def train_bpn(X_train, Y_train, input_size, hidden_size, output_size, epochs, learning_rate):
    """Trains the neural network model using forward and backward propagation."""
    # Initialize weights and biases
    W1, b1, W2, b2 = initialize_weights_bpn(input_size, hidden_size, output_size)

    # Training loop
    for epoch in range(epochs):
        # Forward pass: compute predictions
        Z1, A1, Z2, A2 = forward_propagation_bpn(X_train, W1, b1, W2, b2)

        # Compute the loss using cross-entropy
        loss = cross_entropy_loss(Y_train, A2)

        # Backward pass: compute gradients
        dW1, db1, dW2, db2 = backward_propagation_bpn(X_train, Y_train, Z1, A1, A2, W2)

        # Update weights and biases using gradient descent
        W1 -= learning_rate * dW1  # Update weights for the input to hidden layer
        b1 -= learning_rate * db1  # Update biases for the hidden layer
        W2 -= learning_rate * dW2  # Update weights for the hidden to output layer
        b2 -= learning_rate * db2  # Update biases for the output layer

        # Print the loss every 10 epochs or at the first epoch
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    return W1, b1, W2, b2  # Return the trained weights and biases

# Prediction function
def predict_bpn(X, W1, b1, W2, b2):
    """Makes predictions using the trained model."""
    _, _, _, A2 = forward_propagation_bpn(X, W1, b1, W2, b2)  # Perform forward propagation
    return np.argmax(A2, axis=1)  # Return the index of the highest probability (the predicted class)

# Example usage
if __name__ == "__main__":
    # Define data dimensions
    input_size = 1024  # Number of features (input layer size)
    hidden_size = 512  # Number of neurons in the hidden layer
    output_size = 3  # Number of classes (Alzheimer disease classification with 3 classes)

    # Synthetic data for testing (replace with actual data)
    X_train = result  # Training data
    Y_train = y_train  # One-hot encoded labels for training data

    # Hyperparameters for training
    epochs = 50  # Number of epochs for training
    learning_rate = 0.01  # Learning rate for weight updates

    # Train the BPN model
    W1_bpn, b1_bpn, W2_bpn, b2_bpn = train_bpn(X_train, Y_train, input_size, hidden_size, output_size, epochs, learning_rate)

    # Make predictions on the training data (or test data)
    predictions_bpn = predict_bpn(X_train, W1_bpn, b1_bpn, W2_bpn, b2_bpn)

    # Calculate accuracy by comparing predicted classes with actual labels
    accuracy_bpn = np.mean(predictions_bpn == y_train_new)  # y_train_new contains the true labels
    print(f"BPN Training Accuracy: {accuracy_bpn:.4f}")  # Output the accuracy


Epoch 1/50, Loss: 0.6783
Epoch 10/50, Loss: 1.0620
Epoch 20/50, Loss: 1.0158
Epoch 30/50, Loss: 0.9996
Epoch 40/50, Loss: 0.9795
Epoch 50/50, Loss: 1.0301
BPN Training Accuracy: 0.4955


In [ ]:
# Function to predict class labels
def predict_bpn(X):
    # Forward pass
    z1 = np.dot(X, W1) + b1  # Hidden layer input
    a1 = sigmoid(z1)  # Hidden layer output (Sigmoid activation)
    z2 = np.dot(a1, W2) + b2  # Output layer input
    a2 = sigmoid(z2)  # Output layer output (Sigmoid activation)
    return a2

# Function to calculate accuracy
def calculate_accuracy_bpn(X_test, Y_test):
    # Get predictions
    predictions = predict_bpn(X_test)
    
    # Convert predictions to class labels
    predicted_classes = np.argAmax(predictions, axis=1)  # Choose the class with the highest probability
    true_classes = np.argmax(Y_test, axis=1)  # Convert one-hot encoded labels to class labels
    
    # Calculate accuracy
    accuracy = np.mean(predicted_classes == true_classes)  # Percentage of correct predictions
    return accuracy

# Example usage:
accuracy_bpn = calculate_accuracy_bpn(x_test_flat, y_test_data1)  # X_test_flat and y_test are your testing data and labels
print(f"Accuracy for Pure BPN: {accuracy_bpn * 100:.2f}%")


Accuracy for Pure BPN: 22.71%
